In [1]:
from openff.toolkit import Molecule
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from rdkit.Chem.AllChem import ComputeMolVolume
import math
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from openff.toolkit import ForceField, Molecule, Topology
import pathlib
import logging

In [2]:
logging.getLogger("openff").setLevel(logging.ERROR)

# Smiles of the solutes and solvents picked by Melissa

In [3]:
#original set
solutelstsmiles = ['O=Cc1cc(Br)c(O)c(Br)c1', 'Cc1cc(C(=O)O)ccc1[N+](=O)[O-]', 'Oc1ccc(OCc2ccccc2)cc1', 'Nc1ccc(S(=O)(=O)Nc2ccc(Cl)nn2)cc1', 'CC(=O)O[C@]1(C(C)=O)CC[C@H]2[C@@H]3C=C(Cl)C4=CC(=O)[C@@H]5C[C@@H]5[C@]4(C)[C@H]3CC[C@@]21C', 'C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@@](C)(C(=O)O1)[C@H]2[C@@H]3C(=O)O', 'N#CCc1ccc([N+](=O)[O-])cc1', 'O=C(O)c1cc([N+](=O)[O-])cc([N+](=O)[O-])c1O', 'N#Cc1cccc([N+](=O)[O-])c1', 'CC(=O)N[C@@H](CC(C)C)C(=O)O', 'CC(=O)N[C@@H](CC(C)C)C(=O)O', 'N[C@@H](CO)C(=O)O', 'COc1cc([N+](=O)[O-])ccc1N', 'Nc1c(Cl)cc([N+](=O)[O-])cc1Cl', 'CC(=O)Nc1ccccc1[N+](=O)[O-]', 'N[C@H](Cc1c[nH]c2ccccc12)C(=O)O', 'Cl.N[C@@H](Cc1ccccc1)C(=O)OCc1ccccc1', 'CC(=O)N1CCC[C@H]1C(=O)O', 'CSCCC(NC(C)=O)C(=O)O', 'CCOC(=O)[C@@H]1CSCN1.Cl', 'OC[C@H]1O[C@@H](Oc2ccc(O)cc2)[C@H](O)[C@@H](O)[C@@H]1O', 'OCc1ccccc1O[C@@H]1OC(CO)[C@@H](O)[C@@H](O)C1O', 'CC(=O)Nc1ccc(OC(=O)c2ccccc2OC(C)=O)cc1', 'COc1ccc(CN2CCNCC2)c(OC)c1OC.Cl', 'CC(=O)Oc1ccccc1C(=O)Nc1ncc([N+](=O)[O-])s1', 'COc1c(N2CCNC(C)C2)c(F)cc2c(=O)c(C(=O)O)cn(C3CC3)c12', 'CN1CCN(c2c(F)cc3c(=O)c(C(=O)O)cn4c3c2OCN4C)CC1']
solventlstsmiles = ['CCO', 'CCCCO', 'CCOC(C)=O', 'CC#N', 'CC(C)O']
# ['ethanol', 'n-butanol', 'ethyl acetate', 'acetonitrile', 'isopropanol']

# Check to filter out any salts 

In [4]:
newsolutelstsmiles = []
for solute in solutelstsmiles:
    if '.' not in solute:
        newsolutelstsmiles.append(solute)

In [5]:
from rdkit.Chem.Descriptors import ExactMolWt
def logsconv(molefraction, smilestring):
    mol = Chem.MolFromSmiles(smilestring)
    molarmass = ExactMolWt(mol)
    molarmasskg = molarmass * 0.001
    molfractsolute = molefraction
    molfractsolvent = 1 - molfractsolute
    h20density = 997 #density of water at room temp in Kilogram per Cubic Meter
    h20mass = 0.01801528 #molar mass of water at room temp in kg
    sval = ((molfractsolute*h20density*0.001)/(molfractsolvent*h20mass+molfractsolute*molarmasskg))
    logsval = math.log10(sval)
    return(logsval)

# Creating a dataframe of solute+solvent pairs

In [6]:
df30315 = pd.read_csv('df30315.csv')

solrangedf = df30315.copy()

count = 0
for i in solrangedf['SMILES_Solvent']:
    if i not in solventlstsmiles:
        solrangedf = solrangedf.drop(solrangedf.index[count])
        count -= 1
    count += 1
count = 0
for i in solrangedf['SMILES']:
    if i not in newsolutelstsmiles:
        solrangedf = solrangedf.drop(solrangedf.index[count])
        count -= 1
    count += 1

solubilitylst = []
logslst = []
for i in solrangedf['Solubility']:
    solubilitylst.append(i)
smileslst = []
for i in solrangedf['SMILES']:
    smileslst.append(i)
for i,j in zip(solubilitylst, smileslst):
    logslst.append(logsconv(i,j))
print(len(logslst))
solrangedf['LogS'] = logslst

solvlst = ['ethanol', 'n-butanol', 'ethyl acetate', 'acetonitrile', 'isopropanol']

mol1df = solrangedf.copy()
count = 0
for i in mol1df['Solvent']:
    if i not in solvlst[4]:
        mol1df = mol1df.drop(mol1df.index[count])
        count -= 1
    count += 1
mol1df = mol1df.drop_duplicates(subset=["SMILES"], keep='first')
templst = list(mol1df['T,K'])
smileslst = list(mol1df['SMILES'])


136


In [7]:
solvdensity = [0.789, 0.801, 0.902, 0.786, 0.786]
solv_dict = {'ethanol':['0.789g/cm^3', '732.40-866.84kg/m^3 at 278.15-348.15K'],
             'n-butanol':['0.801g/cm^3', '752.08-817.21kg/m^3 at 283.15-363.15K'],
             'ethyl acetate':['0.902g/cm^3', '830.62-906.55kg/m^3 at 288.15-348.15K'],
             'acetonitrile':['0.786g/cm^3', '726.43-798.12kg/m^3 at 278.15-343.15K'],
             'isopropanol': ['0.786g/cm^3', '738.91-810.90kg/m^3 at 263.10-343.30K']}

In [8]:
all_solvent_df = solrangedf.copy()
parent_dir = "./abcg2"
dirnum = 1
for item in solvlst:
    df = all_solvent_df[all_solvent_df['Solvent'] == item]
    df = df.drop_duplicates(subset=["SMILES"], keep='first')
    df = df.reset_index()
    missinglst = list(df['SMILES'])
    print(df.head())
    print(len(df))

    
    for i in range(len(df)):
        #create solvent.sdf
        mol_solvent = Molecule.from_smiles(df['SMILES_Solvent'].iloc[i], allow_undefined_stereo=True)
        mol_solvent.generate_conformers()
        mol_solvent.to_file("solvent.sdf", file_format="sdf")
        #create solute.sdf
        mol_solute = Molecule.from_smiles(df['SMILES'].iloc[i], allow_undefined_stereo=True)
        mol_solute.generate_conformers()
        mol_solute.to_file("solute.sdf", file_format="sdf")
        #create readme.md file
        mdFile = open('README', 'w')
        mdFile.write("Solute: % s \n" % df['SMILES'].iloc[i])
        mdFile.write("Solvent: % s \n" % df['SMILES_Solvent'].iloc[i])
        mdFile.write("Solvent Name: % s \n" % df['Solvent'].iloc[i])
        mdFile.write(f"Solvent Density at Room Temp: {solv_dict[df['Solvent'].iloc[i]][0]} \n") 
        mdFile.write(f"Solvent Density: {solv_dict[df['Solvent'].iloc[i]][1]} \n") 
        mdFile.write("Pressure: 1.01325 atm \n")
        mdFile.write("Temperature: % sK \n" % df['T,K'].iloc[i])
        mdFile.write("Solubility: % s (mole fraction) \n" % df['Solubility'].iloc[i])
        mdFile.write("LogS: % s \n" % df['LogS'].iloc[i])
        mdFile.write("DOI: % s \n" % df['Source'].iloc[i])
        mdFile.close()
        #make new directory
        directory = str(dirnum).zfill(4)
        path = os.path.join(parent_dir, directory)
        pathlib.Path(path).mkdir(parents=True, exist_ok=True) 
        # print("Directory '% s' created" % directory)
        #place files in correct directory
        os.rename("solvent.sdf", "./"+parent_dir+"/% s/solvent.sdf" % directory)
        os.rename("solute.sdf", "./"+parent_dir+"/% s/solute.sdf" % directory)
        os.rename("README", "./"+parent_dir+"/% s/README.md" % directory)
        dirnum += 1


   index                                      SMILES     T,K  Solubility  \
0    358  CC(=O)Oc1ccccc1C(=O)Nc1ncc([N+](=O)[O-])s1  303.15    0.000331   
1    359                      O=Cc1cc(Br)c(O)c(Br)c1  303.15    0.005547   
2    360      CC(=O)Nc1ccc(OC(=O)c2ccccc2OC(C)=O)cc1  303.15    0.000486   
3    372                 CC(=O)Nc1ccccc1[N+](=O)[O-]  303.15    0.032350   
4    373                        CSCCC(NC(C)=O)C(=O)O  303.15    0.095400   

   Solvent SMILES_Solvent                    Source      LogS  
0  ethanol            CCO  10.1021/acs.jced.0c00254 -1.739418  
1  ethanol            CCO  10.1021/acs.jced.0c00376 -0.546314  
2  ethanol            CCO  10.1021/acs.jced.0c00301 -1.573753  
3  ethanol            CCO  10.1021/acs.jced.1c00037  0.142012  
4  ethanol            CCO  10.1021/acs.jced.1c00064  0.440124  
23


   index                                      SMILES     T,K  Solubility  \
0   1058  CC(=O)Oc1ccccc1C(=O)Nc1ncc([N+](=O)[O-])s1  303.15    0.000471   
1   1059                      O=Cc1cc(Br)c(O)c(Br)c1  303.15    0.008456   
2   1060      CC(=O)Nc1ccc(OC(=O)c2ccccc2OC(C)=O)cc1  303.15    0.000557   
3   1070                 CC(=O)Nc1ccccc1[N+](=O)[O-]  303.15    0.023210   
4   1071                        CSCCC(NC(C)=O)C(=O)O  303.15    0.059100   

     Solvent SMILES_Solvent                    Source      LogS  
0  n-butanol          CCCCO  10.1021/acs.jced.0c00254 -1.587194  
1  n-butanol          CCCCO  10.1021/acs.jced.0c00376 -0.379760  
2  n-butanol          CCCCO  10.1021/acs.jced.0c00301 -1.515035  
3  n-butanol          CCCCO  10.1021/acs.jced.1c00037  0.026388  
4  n-butanol          CCCCO  10.1021/acs.jced.1c00064  0.319382  
23


   index                                      SMILES     T,K  Solubility  \
0   1568  CC(=O)Oc1ccccc1C(=O)Nc1ncc([N+](=O)[O-])s1  303.15    0.003298   
1   1569                      O=Cc1cc(Br)c(O)c(Br)c1  303.15    0.011390   
2   1570      CC(=O)Nc1ccc(OC(=O)c2ccccc2OC(C)=O)cc1  303.15    0.001189   
3   1581                 CC(=O)Nc1ccccc1[N+](=O)[O-]  303.15    0.112000   
4   1582                        CSCCC(NC(C)=O)C(=O)O  303.15    0.006410   

         Solvent SMILES_Solvent                    Source      LogS  
0  ethyl acetate      CCOC(C)=O  10.1021/acs.jced.0c00254 -0.761086  
1  ethyl acetate      CCOC(C)=O  10.1021/acs.jced.0c00376 -0.266481  
2  ethyl acetate      CCOC(C)=O  10.1021/acs.jced.0c00301 -1.190141  
3  ethyl acetate      CCOC(C)=O  10.1021/acs.jced.1c00037  0.489642  
4  ethyl acetate      CCOC(C)=O  10.1021/acs.jced.1c00064 -0.476037  
23


   index                                      SMILES     T,K  Solubility  \
0   1903  CC(=O)Oc1ccccc1C(=O)Nc1ncc([N+](=O)[O-])s1  303.15    0.001576   
1   1904                      O=Cc1cc(Br)c(O)c(Br)c1  303.15    0.004517   
2   1905      CC(=O)Nc1ccc(OC(=O)c2ccccc2OC(C)=O)cc1  303.15    0.001994   
3   1914                 CC(=O)Nc1ccccc1[N+](=O)[O-]  303.15    0.038690   
4   1915                        CSCCC(NC(C)=O)C(=O)O  303.15    0.008770   

        Solvent SMILES_Solvent                    Source      LogS  
0  acetonitrile           CC#N  10.1021/acs.jced.0c00254 -1.070233  
1  acetonitrile           CC#N  10.1021/acs.jced.0c00376 -0.629507  
2  acetonitrile           CC#N  10.1021/acs.jced.0c00301 -0.971178  
3  acetonitrile           CC#N  10.1021/acs.jced.1c00037  0.200964  
4  acetonitrile           CC#N  10.1021/acs.jced.1c00064 -0.349072  
23


   index                                      SMILES     T,K  Solubility  \
0   2140  CC(=O)Oc1ccccc1C(=O)Nc1ncc([N+](=O)[O-])s1  303.15    0.000436   
1   2141                      O=Cc1cc(Br)c(O)c(Br)c1  303.15    0.006290   
2   2142      CC(=O)Nc1ccc(OC(=O)c2ccccc2OC(C)=O)cc1  303.15    0.000262   
3   2152                 CC(=O)Nc1ccccc1[N+](=O)[O-]  303.15    0.018400   
4   2153                        CSCCC(NC(C)=O)C(=O)O  303.15    0.066010   

       Solvent SMILES_Solvent                    Source      LogS  
0  isopropanol         CC(C)O  10.1021/acs.jced.0c00254 -1.620486  
1  isopropanol         CC(C)O  10.1021/acs.jced.0c00376 -0.496010  
2  isopropanol         CC(C)O  10.1021/acs.jced.0c00301 -1.840504  
3  isopropanol         CC(C)O  10.1021/acs.jced.1c00037 -0.058640  
4  isopropanol         CC(C)O  10.1021/acs.jced.1c00064  0.349396  
23


In [9]:
#copying files from directory to others
mdp_dir = './template_mdp_files/'
mdpfiles = ['em.X.mdp', 'nvt.X.mdp', 'npt.X.mdp',  'prod.X.mdp']
files_to_copy = ['em.X.mdp', 'npt.X.mdp', 'nvt.X.mdp', 'prod.X.mdp', 'run_eq.sh', 'run_hrex.sh']
files_to_copy = [mdp_dir + fle for fle in files_to_copy]

for i in range(1,116): # hardcoded the # of solute solvent pairs 28 * 5
    destination_directory = "./"+parent_dir+"/% s" % str(i).zfill(4)
    for file_to_copy in files_to_copy:
        shutil.copy(file_to_copy, destination_directory)
    
    
    # Excerpt from Meghan's code to create the lambda_directories
    lambda_windows=20
    outdir = "./"+parent_dir+"/% s" % str(i).zfill(4)+'/complete_files'
    

    if not os.path.isdir(outdir): os.mkdir(outdir)
    for j in range(0, lambda_windows):
        # Copy and edit mdp files
        for mdpfile in mdpfiles:
            mdpfilename = os.path.join( outdir, mdpfile.replace('.X.', '.%s.' % j))
            shutil.copy("./"+parent_dir+"/% s" % str(i).zfill(4)+'/'+mdpfile, mdpfilename)
            file = open(mdpfilename, 'r')
            text = file.readlines()
            file.close()
            file = open(mdpfilename, 'w')
            for idx, line in enumerate(text):
               if 'init_lambda_state' in line and line[0] != ';':
                     line = 'init_lambda_state        = %s\n' % j
               file.write(line)
            file.close()


# Using ABCG2 Charge method

In [10]:
# Since AmberTools version 24 "ABCG2" charge method is available, and can be accessed 
# by adding it to openff-toolkit's antechamber supported charge methods

from openff.toolkit.utils.ambertools_wrapper import AmberToolsToolkitWrapper
AmberToolsToolkitWrapper.SUPPORTED_CHARGE_METHODS.update({'abcg2': {'antechamber_keyword': 'abcg2',
  'min_confs': 1,
  'max_confs': 1,
  'rec_confs': 1}})

In [11]:
from openff.interchange.components._packmol import pack_box, RHOMBIC_DODECAHEDRON
from openff.toolkit import Molecule, ForceField
from openff.units import unit
import numpy as np
from openff.interchange import Interchange
import os

_G_PER_ML = unit.grams / unit.milliliters
ff_name = 'openff_unconstrained-2.2.0.offxml'

molcount = 1
solvnum = 0
#calculating number of copies
for solvent in solventlstsmiles:
    solvmol = Molecule.from_smiles(solvent, allow_undefined_stereo=True)
    solvmol.generate_conformers()
    solvrdmol = solvmol.to_rdkit()
    solvmolmatrix = Chem.rdmolops.Get3DDistanceMatrix(solvrdmol)
    den = solvdensity[solvnum]
    solvnum += 1
    for i in range(len(df)):
        solute = df['SMILES'].iloc[i]
        mol = Molecule.from_smiles(solute, allow_undefined_stereo=True)
        mol.generate_conformers()
        ##################################################################
        ##################################################################
        # THIS BLOCK IS CRITICAL for MH's project
        mol1 = Molecule.from_smiles(solute, allow_undefined_stereo=True)
        mol2 = Molecule.from_smiles(solvent, allow_undefined_stereo=True)
        
        mol1.generate_conformers(n_conformers=1)
        mol2.generate_conformers(n_conformers=1)
        
        mol1.assign_partial_charges(partial_charge_method='abcg2')
        mol2.assign_partial_charges(partial_charge_method='abcg2')

        
        off_top = pack_box(molecules=[mol2], 
                           number_of_copies=[400], 
                           target_density=den*0.9*_G_PER_ML, 
                           solute=mol1.to_topology(),
                           center_solute=True, 
                           box_shape=RHOMBIC_DODECAHEDRON)
        sage = ForceField(ff_name)
        out = Interchange.from_smirnoff(force_field=sage, topology=off_top, charge_from_molecules=set(list(off_top.molecules)))
        ##################################################################
        ##################################################################
        out.to_gro('% s.gro' % str(molcount).zfill(4))
        out.to_top('% s.top' % str(molcount).zfill(4))
        
        #place files in correct directory
        source_path = "./%s.gro" % str(molcount).zfill(4)
        destination_path = "./"+parent_dir+"/%s/%s.gro" % (str(molcount).zfill(4), str(molcount).zfill(4))
        os.rename(source_path, destination_path)
        source_path = "./%s.top" % str(molcount).zfill(4)
        destination_path = "./"+parent_dir+"/%s/%s.top" % (str(molcount).zfill(4), str(molcount).zfill(4))
        os.rename(source_path, destination_path)
        molcount += 1
        break
    break